In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pypsa

from pathlib import Path

In [9]:
outdir = Path.cwd() / 'example_outputs'

fn = "network_s_nodal_solved.nc"
datadir = '{}_{}'
root = Path.cwd().parent / "resources" / "live_data"

In [12]:
gpd.read_file(root / datadir.format("2024-03-07", 4) / "regions_offshore_s.geojson").head()

,name,geometry
0,4956,"POLYGON ((2.35762 51.70601, 1.14430 51.65117, ..."
1,4969,"MULTIPOLYGON (((0.58403 51.38764, 0.55945 51.4..."
2,4970,"POLYGON ((0.80502 51.35562, 0.62994 51.38098, ..."
3,4971,"POLYGON ((0.09991 50.76810, 0.24222 50.73805, ..."
4,4974,"POLYGON ((1.14430 51.65117, 1.01538 51.36001, ..."


In [27]:
def fix_snaps(df, date, period):
    df.index = [pd.Timestamp(date) + (period - 1) * pd.Timedelta(f'30min')]
    return df

pnoms = pd.DataFrame()
p = pd.DataFrame()
psets = pd.DataFrame()
line_loads = pd.DataFrame()
link_loads = pd.DataFrame()
prices = pd.DataFrame()

date = '2024-03-07'

for period in range(1, 49):

    n = pypsa.Network(root / datadir.format(date, period) / fn)    
    time = pd.Timestamp(date) + (period - 1) * pd.Timedelta(f'30min')

    pnoms = pd.concat((pnoms, n.generators[["p_nom"]].rename(columns={"p_nom": time})), axis=1)
    p = pd.concat((p, fix_snaps(n.generators_t["p"], date, period)), axis=0)
    line_loads = pd.concat((line_loads, fix_snaps(n.lines_t["p0"], date, period)), axis=0)
    link_loads = pd.concat((link_loads, fix_snaps(n.links_t["p0"], date, period)), axis=0)
    prices = pd.concat((prices, fix_snaps(n.buses_t["marginal_price"], date, period)), axis=0)

    psets = pd.concat((psets, n.generators[["p_set"]].rename(columns={"p_set": time})), axis=1)



INFO:pypsa.io:Imported network network_s_nodal_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_nodal_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_nodal_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_nodal_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_nodal_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_nodal_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_nodal_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_nodal_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_nodal_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa

In [28]:
psets.T.to_json(outdir / "psets.json")
pnoms.T.to_json(outdir / "pnoms.json")
prices.to_json(outdir / "prices.json")
line_loads.to_json(outdir / "line_loads.json")
link_loads.to_json(outdir / "link_loads.json")
p.to_json(outdir / "p.json")

In [19]:
n = pypsa.Network(root / datadir.format("2024-03-07", 4) / fn)

n.generators.to_json(outdir / "generators_static.json")
n.buses.to_json(outdir / "buses_static.json")
n.lines.to_json(outdir / "lines_static.json")
n.links.to_json(outdir / "links_static.json")
n.loads.to_json(outdir / "loads_static.json")

INFO:pypsa.io:Imported network network_s_nodal_solved.nc has buses, carriers, generators, lines, links, loads
